In [1]:
# SLA Predictor application
# CLASS Project: https://class-project.eu/

#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
#
# Created on 25 Mar 2021
# @author: Jorge Montero - ATOS
#

# prometheus_api_client

In [2]:
from prometheus_api_client import PrometheusConnect
prom = PrometheusConnect(url ="http://192.168.7.42:9091/", disable_ssl=True)

# Get the list of all the metrics that the Prometheus host scrapes
prom.all_metrics()

['deadlines_missed_0fn5jwwiuvnoj6smtfwk7u1614155040086017752',
 'deadlines_missed_0j6apqscnzd2pfst2hktfc1615372473841624998',
 'deadlines_missed_0rrkko98mqwnmxsceme2x41614601459187461562',
 'deadlines_missed_15dvgrrvvdsm9xmxcta7md1614004570092499581',
 'deadlines_missed_1qikxbsn3dnjsetcr8wyqf1614004893766124876',
 'deadlines_missed_1qnf41hn9kmhi32fwup0zg1612543146431620328',
 'deadlines_missed_1zrbnoporm44hg1rusa2iv1614006681162815415',
 'deadlines_missed_1zs6xnmelt1oe9jsfw5edx1614589304010498708',
 'deadlines_missed_252yyxfxtp6fkjxnthjgxl1614071503322899071',
 'deadlines_missed_37kuwnhjgfz8uhjun7d2b11614588612274250339',
 'deadlines_missed_3vpkbo6hov6euumavbqhfa1614585635341454788',
 'deadlines_missed_3xbztukcovhfleoe0qfhbj1614158096406654380',
 'deadlines_missed_6bjfx2kguvjr337m58v55f1614248755678305737',
 'deadlines_missed_6dw1pwexfossssofs7tyhk1615297278605891440',
 'deadlines_missed_77039ip9uv8b2bntls4iky1615298033775262248',
 'deadlines_missed_7mfwrw1us1amlmptaxbvhf16141535726286

# Training data generation

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [4]:
metrics_names = ['go_goroutines','go_memstats_alloc_bytes','go_memstats_gc_cpu_fraction',
'go_memstats_gc_sys_bytes', 'go_memstats_heap_alloc_bytes',
'go_memstats_heap_idle_bytes', 'go_memstats_heap_inuse_bytes',
'go_memstats_heap_objects', 'go_memstats_heap_released_bytes',
'go_memstats_heap_sys_bytes', 'go_memstats_last_gc_time_seconds',
'go_memstats_mspan_inuse_bytes', 'go_memstats_next_gc_bytes',
'go_memstats_other_sys_bytes','go_memstats_stack_inuse_bytes',
'go_memstats_stack_sys_bytes', 'go_threads', 'node_boot_time_seconds',
'node_entropy_available_bits', 'node_filefd_allocated' ,'node_load1',
'node_load15', 'node_load5', 'node_memory_Active_anon_bytes',
'node_memory_Active_bytes', 'node_memory_Active_file_bytes',
'node_memory_AnonHugePages_bytes', 'node_memory_AnonPages_bytes',
'node_memory_Buffers_bytes', 'node_memory_Cached_bytes',
'node_memory_Committed_AS_bytes', 'node_memory_DirectMap2M_bytes',
'node_memory_DirectMap4k_bytes', 'node_memory_Dirty_bytes',
'node_memory_Inactive_anon_bytes', 'node_memory_Inactive_bytes',
'node_memory_Inactive_file_bytes', 'node_memory_KernelStack_bytes',
'node_memory_Mapped_bytes', 'node_memory_MemAvailable_bytes',
'node_memory_MemFree_bytes', 'node_memory_PageTables_bytes',
'node_memory_SReclaimable_bytes', 'node_memory_SUnreclaim_bytes',
'node_memory_Shmem_bytes', 'node_memory_Slab_bytes', 'node_procs_running', 'node_sockstat_TCP_alloc',
'node_sockstat_TCP_mem', 'node_sockstat_TCP_mem_bytes',
'node_sockstat_sockets_used', 'node_time_seconds',
'node_timex_frequency_adjustment_ratio', 'node_timex_maxerror_seconds',
'node_timex_offset_seconds', 'process_resident_memory_bytes',
'process_start_time_seconds']

Funtion to aggregate all the instances data of the timeseries for each metric

In [5]:
def get_timeseries_from_metric(metric_name,start_time,end_time,chunk_size):
    metric_data = prom.get_metric_range_data(
        metric_name,  # this is the metric name and label config
        start_time=start_time,
        end_time=end_time,
        chunk_size=chunk_size,
    )
    
    # do some process to it: merging all timeseries values to one, and get the aggregated value
    metric_d_all_df = pd.DataFrame()
    if metric_data:
        for i in range(0,len(metric_data)):
            metric_d_df = pd.DataFrame(metric_data[i]["values"],columns=["timestamp", metric_name+str(i)])
            metric_d_df['timestamp']= pd.to_datetime(metric_d_df['timestamp'], unit='s')
            metric_d_df[metric_name+str(i)]= pd.to_numeric(metric_d_df[metric_name+str(i)], errors='coerce')
            metric_d_df.set_index('timestamp', inplace=True)

            metric_d_all_df = pd.concat([metric_d_all_df, metric_d_df], axis=0)

        #metric_d_all_df = metric_d_all_df.groupby(pd.Grouper(freq='1Min')).aggregate("last")

        metric_d_agg_df = metric_d_all_df
        metric_d_agg_df[metric_name] = metric_d_all_df.aggregate("mean", axis=1)
        #return metric_d_agg_df[metric_name]     

        metric_data_insert = []
        metric_data_insert_time = metric_d_agg_df.index.values
        metric_data_insert_val = metric_d_agg_df[metric_name].values
        for i in range(0,len(metric_data_insert_time)):
            metric_data_insert.append([metric_data_insert_time[i],metric_data_insert_val[i]])
        metric_data_df = pd.DataFrame(metric_data_insert,columns=["timestamp", metric_name])
        metric_data_df['timestamp']= pd.to_datetime(metric_data_df['timestamp'], unit='s')
        metric_data_df[metric_name]= pd.to_numeric(metric_data_df[metric_name], errors='coerce')
        metric_data_df.set_index('timestamp', inplace=True)
        return metric_data_df
    
    else:
        return pd.DataFrame()

In [6]:
def historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names):    
    metrics_all_df = pd.DataFrame()
    
    print(end_time)
    for metric_name in metrics_names:
        print(metric_name)
        metric_data_df = get_timeseries_from_metric(metric_name,start_time,end_time,chunk_size)
        #print(metric_data_df)
        if not metric_data_df.empty:
            metrics_all_df = pd.concat([metrics_all_df, metric_data_df], axis=0)
            
    metrics_all_df = metrics_all_df.groupby(pd.Grouper(freq='1Min')).aggregate("last")
    
    print("-------------------------------------")
    return metrics_all_df

Define the 40 executions and retrieve the data for each execution

In [7]:
from prometheus_api_client.utils import parse_datetime
from datetime import timedelta

# added 1 hour to match local time


#{instance="violations",job="sla"}

#--------------------
#2021/02/22
#--------------------
#T1 (15:10)
#execution_time_1zrbnoporm44hg1rusa2iv1614006681162815415     617820
#workers_1zrbnoporm44hg1rusa2iv1614006681162815415     3

start_time = parse_datetime("2021-02-22 15:10:00")
end_time = parse_datetime("2021-02-22 16:10:00")
chunk_size = timedelta(minutes=10)

metrics_T1 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)
    
#T2 (15:29)
#execution_time_uu32hapfrhxk8die5nghfp1614007776284785881 571378
#workers_uu32hapfrhxk8die5nghfp1614007776284785881     3

start_time = parse_datetime("2021-02-22 15:29:00")
end_time = parse_datetime("2021-02-22 16:29:00")
chunk_size = timedelta(minutes=10)

metrics_T2 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T3 (16:11)
#execution_time_sjrzkf4amydnk3ysej5ep41614010268217036106 566439
#workers_sjrzkf4amydnk3ysej5ep41614010268217036106     3

start_time = parse_datetime("2021-02-22 16:11:00")
end_time = parse_datetime("2021-02-22 17:11:00")
chunk_size = timedelta(minutes=10)

metrics_T3 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T4 (16:24)
#execution_time_mtzdhikevn1ud5kwpjc321614011066708126874 564199
#workers_mtzdhikevn1ud5kwpjc321614011066708126874     3

start_time = parse_datetime("2021-02-22 16:24:00")
end_time = parse_datetime("2021-02-22 17:24:00")
chunk_size = timedelta(minutes=10)

metrics_T4 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#--------------------
#2021/02/23
#--------------------
#T5 (09:12)
#execution_time_252yyxfxtp6fkjxnthjgxl1614071503322899071     578790
#workers_252yyxfxtp6fkjxnthjgxl1614071503322899071     3

start_time = parse_datetime("2021-02-23 09:12:00")
end_time = parse_datetime("2021-02-23 10:12:00")
chunk_size = timedelta(minutes=10)

metrics_T5 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T6 (09:25)
#execution_time_bqvqdvp491gurrcutqqxsj1614072298139170395     567978
#workers_bqvqdvp491gurrcutqqxsj1614072298139170395     3

start_time = parse_datetime("2021-02-23 09:25:00")
end_time = parse_datetime("2021-02-23 10:25:00")
chunk_size = timedelta(minutes=10)

metrics_T6 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T7 (09:50)
#execution_time_auszuvzbq101vxkq0bfncb1614073802185063278     565619
#workers_auszuvzbq101vxkq0bfncb1614073802185063278     3

start_time = parse_datetime("2021-02-23 09:50:00")
end_time = parse_datetime("2021-02-23 10:50:00")
chunk_size = timedelta(minutes=10)

metrics_T7 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T8 (10:23)
#execution_time_wdxsqwpenmsr0ywnchrptu1614075789610887034     565052
#workers_wdxsqwpenmsr0ywnchrptu1614075789610887034     3

start_time = parse_datetime("2021-02-23 10:23:00")
end_time = parse_datetime("2021-02-23 11:23:00")
chunk_size = timedelta(minutes=10)

metrics_T8 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T9 (10:40)
#execution_time_k5s5zw6l6r5hnbe8q9muq41614076798006104354     572151
#workers_k5s5zw6l6r5hnbe8q9muq41614076798006104354     3

start_time = parse_datetime("2021-02-23 10:40:00")
end_time = parse_datetime("2021-02-23 11:40:00")
chunk_size = timedelta(minutes=10)

metrics_T9 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T10 (10:54)
#execution_time_t41to1z665sbrpmy6h8ef51614077652490912372     582628
#workers_t41to1z665sbrpmy6h8ef51614077652490912372     3

start_time = parse_datetime("2021-02-23 10:54:00")
end_time = parse_datetime("2021-02-23 11:54:00")
chunk_size = timedelta(minutes=10)

metrics_T10 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T11 (11:09)
#execution_time_wqxsdmfoh1ugde4es0g80d1614078548341014474     636311
#workers_wqxsdmfoh1ugde4es0g80d1614078548341014474     3

start_time = parse_datetime("2021-02-23 11:09:00")
end_time = parse_datetime("2021-02-23 12:09:00")
chunk_size = timedelta(minutes=10)

metrics_T11 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T12 (11:34)
#execution_time_ynfk8xpf0j2bjhu5l97scl1614080036792723185     593063
#workers_ynfk8xpf0j2bjhu5l97scl1614080036792723185     3

start_time = parse_datetime("2021-02-23 11:34:00")
end_time = parse_datetime("2021-02-23 12:34:00")
chunk_size = timedelta(minutes=10)

metrics_T12 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T13 (11:53)
#execution_time_rrifturxkfkxmdidwsjjv21614081204191835454     522618
#workers_rrifturxkfkxmdidwsjjv21614081204191835454     6

start_time = parse_datetime("2021-02-23 11:53:00")
end_time = parse_datetime("2021-02-23 12:53:00")
chunk_size = timedelta(minutes=10)

metrics_T13 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#--------------------
#2021/02/24
#--------------------

#T14 (08:00)
#execution_time_7mfwrw1us1amlmptaxbvhf1614153572628637210     454466
#workers_7mfwrw1us1amlmptaxbvhf1614153572628637210     6

start_time = parse_datetime("2021-02-24 08:00:00")
end_time = parse_datetime("2021-02-24 09:00:00")
chunk_size = timedelta(minutes=10)

metrics_T14 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T15 (08:13)
#execution_time_qrngir4ynyq9ijejv2gvn91614154372930668746     464412
#workers_qrngir4ynyq9ijejv2gvn91614154372930668746     6

start_time = parse_datetime("2021-02-24 08:13:00")
end_time = parse_datetime("2021-02-24 09:13:00")
chunk_size = timedelta(minutes=10)

metrics_T15 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T16 (08:24)
#execution_time_0fn5jwwiuvnoj6smtfwk7u1614155040086017752     464266
#workers_0fn5jwwiuvnoj6smtfwk7u1614155040086017752     6

start_time = parse_datetime("2021-02-24 08:24:00")
end_time = parse_datetime("2021-02-24 09:24:00")
chunk_size = timedelta(minutes=10)

metrics_T16 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T17 (08:35)
#execution_time_ds0sff2pjsuw4x0gwvgxe91614155687847422874     455849
#workers_ds0sff2pjsuw4x0gwvgxe91614155687847422874     6

start_time = parse_datetime("2021-02-24 08:35:00")
end_time = parse_datetime("2021-02-24 09:35:00")
chunk_size = timedelta(minutes=10)

metrics_T17 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T18 (08:47)
#execution_time_zhejbkcf9rat8hxyamcdnw1614156458223442302     466922
#workers_zhejbkcf9rat8hxyamcdnw1614156458223442302     6

start_time = parse_datetime("2021-02-24 08:47:00")
end_time = parse_datetime("2021-02-24 09:47:00")
chunk_size = timedelta(minutes=10)

metrics_T18 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T19 (09:02)
#execution_time_apjult5hozqgkwmmngtzaj1614157309794988522     468000
#workers_apjult5hozqgkwmmngtzaj1614157309794988522     6

start_time = parse_datetime("2021-02-24 09:02:00")
end_time = parse_datetime("2021-02-24 10:02:00")
chunk_size = timedelta(minutes=10)

metrics_T19 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T20 (09:15)
#execution_time_3xbztukcovhfleoe0qfhbj1614158096406654380     709955
#workers_3xbztukcovhfleoe0qfhbj1614158096406654380        6

start_time = parse_datetime("2021-02-24 09:15:00")
end_time = parse_datetime("2021-02-24 10:15:00")
chunk_size = timedelta(minutes=10)

metrics_T20 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T21 (09:35)
#execution_time_bwur914wkg6vebeawc7hte1614159291347452376     693223
#workers_bwur914wkg6vebeawc7hte1614159291347452376      6 

start_time = parse_datetime("2021-02-24 09:35:00")
end_time = parse_datetime("2021-02-24 10:35:00")
chunk_size = timedelta(minutes=10)

metrics_T21 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T22 (09:52)
#execution_time_nbuye4csa8mgcmcwvhprrh1614160356928436678     595968
#workers_nbuye4csa8mgcmcwvhprrh1614160356928436678      3

start_time = parse_datetime("2021-02-24 09:52:00")
end_time = parse_datetime("2021-02-24 10:52:00")
chunk_size = timedelta(minutes=10)

metrics_T22 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#--------------------
#2021/02/25
#--------------------

#T23 (10:26)
#execution_time_6bjfx2kguvjr337m58v55f1614248755678305737     463951
#workers_6bjfx2kguvjr337m58v55f1614248755678305737 6

start_time = parse_datetime("2021-02-25 10:26:00")
end_time = parse_datetime("2021-02-25 11:26:00")
chunk_size = timedelta(minutes=10)

metrics_T23 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T24 (10:38)
#execution_time_82mqvf1enp8pflio7lznaa1614249473276461617     455543
#workers_82mqvf1enp8pflio7lznaa1614249473276461617 6

start_time = parse_datetime("2021-02-25 10:38:00")
end_time = parse_datetime("2021-02-25 11:38:00")
chunk_size = timedelta(minutes=10)

metrics_T24 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T25 (10:49)
#execution_time_vbuqvbwgvt0gpmjdw1uvle1614250157216062234     472430
#workers_vbuqvbwgvt0gpmjdw1uvle1614250157216062234 6

start_time = parse_datetime("2021-02-25 10:49:00")
end_time = parse_datetime("2021-02-25 11:49:00")
chunk_size = timedelta(minutes=10)

metrics_T25 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T26 (11:00)
#execution_time_fanz3xvykmp6vgzkozupdg1614250792118975339     459940
#workers_fanz3xvykmp6vgzkozupdg1614250792118975339 6

start_time = parse_datetime("2021-02-25 11:00:00")
end_time = parse_datetime("2021-02-25 12:00:00")
chunk_size = timedelta(minutes=10)

metrics_T26 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T27 (11:15)
#execution_time_geggj9shofjyps6acuctkd1614251723210094413     588759
#workers_geggj9shofjyps6acuctkd1614251723210094413 1

start_time = parse_datetime("2021-02-25 11:15:00")
end_time = parse_datetime("2021-02-25 12:15:00")
chunk_size = timedelta(minutes=10)

metrics_T27 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T28 (11:29)
#execution_time_i6s8uqrbadapage0yh0ebp1614252562172914197     600364
#workers_i6s8uqrbadapage0yh0ebp1614252562172914197 1

start_time = parse_datetime("2021-02-25 11:29:00")
end_time = parse_datetime("2021-02-25 12:29:00")
chunk_size = timedelta(minutes=10)

metrics_T28 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T29 (11:48)
#execution_time_kpan63s8n1mwwxanslhgra1614253693462386372     594230
#workers_kpan63s8n1mwwxanslhgra1614253693462386372 1

start_time = parse_datetime("2021-02-25 11:48:00")
end_time = parse_datetime("2021-02-25 12:48:00")
chunk_size = timedelta(minutes=10)

metrics_T29 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T30 (12:06)
#execution_time_ru4nfxgxfme6kq0t52gzsf1614254769918108924     599054
#workers_ru4nfxgxfme6kq0t52gzsf1614254769918108924 3

start_time = parse_datetime("2021-02-25 12:06:00")
end_time = parse_datetime("2021-02-25 13:06:00")
chunk_size = timedelta(minutes=10)

metrics_T30 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T31 (12:32)
#execution_time_duy9aq3nz6abpf1vsrxpbj1614256239749564349     457041
#workers_duy9aq3nz6abpf1vsrxpbj1614256239749564349 6

start_time = parse_datetime("2021-02-25 12:32:00")
end_time = parse_datetime("2021-02-25 13:32:00")
chunk_size = timedelta(minutes=10)

metrics_T31 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T32 (12:45)
#execution_time_zjjzpskdq59zd7qsdz4dqy1614257093294678341     464953
#workers_zjjzpskdq59zd7qsdz4dqy1614257093294678341 6

start_time = parse_datetime("2021-02-25 12:45:00")
end_time = parse_datetime("2021-02-25 13:45:00")
chunk_size = timedelta(minutes=10)

metrics_T32 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#--------------------
#2021/03/01
#--------------------

#T33 (08:01)
#execution_time_3vpkbo6hov6euumavbqhfa1614585635341454788     407144
#workers_3vpkbo6hov6euumavbqhfa1614585635341454788 9

start_time = parse_datetime("2021-03-01 08:01:00")
end_time = parse_datetime("2021-03-01 09:01:00")
chunk_size = timedelta(minutes=10)

metrics_T33 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T34 (08:12)
#execution_time_h8quohb48ndwdp5hobha8s1614586348774224598     415762
#workers_h8quohb48ndwdp5hobha8s1614586348774224598 9

start_time = parse_datetime("2021-03-01 08:12:00")
end_time = parse_datetime("2021-03-01 09:12:00")
chunk_size = timedelta(minutes=10)

metrics_T34 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T35 (08:24)
#execution_time_9fe2vdbdptqyjkj5ndhtzv1614587059111284764     416799
#workers_9fe2vdbdptqyjkj5ndhtzv1614587059111284764 9

start_time = parse_datetime("2021-03-01 08:24:00")
end_time = parse_datetime("2021-03-01 09:24:00")
chunk_size = timedelta(minutes=10)

metrics_T35 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T36 (08:38)
#execution_time_tmve1annaasp6cdd6fsfxw1614587916002550332     424616
#workers_tmve1annaasp6cdd6fsfxw1614587916002550332 9

start_time = parse_datetime("2021-03-01 08:38:00")
end_time = parse_datetime("2021-03-01 09:38:00")
chunk_size = timedelta(minutes=10)

metrics_T36 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T37 (08:50)
#execution_time_37kuwnhjgfz8uhjun7d2b11614588612274250339     413798
#workers_37kuwnhjgfz8uhjun7d2b11614588612274250339 9

start_time = parse_datetime("2021-03-01 08:50:00")
end_time = parse_datetime("2021-03-01 09:50:00")
chunk_size = timedelta(minutes=10)

metrics_T37 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T38 (09:02)
#execution_time_1zs6xnmelt1oe9jsfw5edx1614589304010498708     416003
#workers_1zs6xnmelt1oe9jsfw5edx1614589304010498708 9

start_time = parse_datetime("2021-03-01 09:02:00")
end_time = parse_datetime("2021-03-01 10:02:00")
chunk_size = timedelta(minutes=10)

metrics_T38 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T39 (09:13)
#execution_time_prdyxbcbsmtylad7oem9re1614589985030313982     411904
#workers_prdyxbcbsmtylad7oem9re1614589985030313982 9

start_time = parse_datetime("2021-03-01 09:13:00")
end_time = parse_datetime("2021-03-01 10:13:00")
chunk_size = timedelta(minutes=10)

metrics_T39 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)

#T40 (09:25)
#execution_time_npmceuu9esn8tvdw8vscxz1614590691113747823     429661
#workers_npmceuu9esn8tvdw8vscxz1614590691113747823 9

start_time = parse_datetime("2021-03-01 09:25:00")
end_time = parse_datetime("2021-03-01 10:25:00")
chunk_size = timedelta(minutes=10)

metrics_T40 = historical_values_for_metrics(start_time, end_time, chunk_size, metrics_names)


2021-02-22 16:10:00
go_goroutines
go_memstats_alloc_bytes
go_memstats_gc_cpu_fraction
go_memstats_gc_sys_bytes
go_memstats_heap_alloc_bytes
go_memstats_heap_idle_bytes
go_memstats_heap_inuse_bytes
go_memstats_heap_objects
go_memstats_heap_released_bytes
go_memstats_heap_sys_bytes
go_memstats_last_gc_time_seconds
go_memstats_mspan_inuse_bytes
go_memstats_next_gc_bytes
go_memstats_other_sys_bytes
go_memstats_stack_inuse_bytes
go_memstats_stack_sys_bytes
go_threads
node_boot_time_seconds
node_entropy_available_bits
node_filefd_allocated
node_load1
node_load15
node_load5
node_memory_Active_anon_bytes
node_memory_Active_bytes
node_memory_Active_file_bytes
node_memory_AnonHugePages_bytes
node_memory_AnonPages_bytes
node_memory_Buffers_bytes
node_memory_Cached_bytes
node_memory_Committed_AS_bytes
node_memory_DirectMap2M_bytes
node_memory_DirectMap4k_bytes
node_memory_Dirty_bytes
node_memory_Inactive_anon_bytes
node_memory_Inactive_bytes
node_memory_Inactive_file_bytes
node_memory_KernelStack_

In [8]:
metrics_T40

,go_goroutines,go_memstats_alloc_bytes,go_memstats_gc_cpu_fraction,go_memstats_gc_sys_bytes,go_memstats_heap_alloc_bytes,go_memstats_heap_idle_bytes,go_memstats_heap_inuse_bytes,go_memstats_heap_objects,go_memstats_heap_released_bytes,go_memstats_heap_sys_bytes,...,node_sockstat_TCP_alloc,node_sockstat_TCP_mem,node_sockstat_TCP_mem_bytes,node_sockstat_sockets_used,node_time_seconds,node_timex_frequency_adjustment_ratio,node_timex_maxerror_seconds,node_timex_offset_seconds,process_resident_memory_bytes,process_start_time_seconds
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-03-01 08:25:00,7.0,1955528.0,0.000015,3582216.0,1955528.0,62857216.0,3399680.0,8111.0,62038016.0,66256896.0,...,78.0,8.0,32768.0,669.0,1.614587e+09,1.000028,1.4645,2.890000e-06,25243648.0,1.612092e+09
2021-03-01 08:26:00,7.0,3516488.0,0.000015,3582216.0,3516488.0,60727296.0,5529600.0,14509.0,59662336.0,66256896.0,...,79.0,3.0,12288.0,670.0,1.614587e+09,1.000028,1.4945,2.570000e-06,25243648.0,1.612092e+09
2021-03-01 08:27:00,7.0,2997928.0,0.000015,3582216.0,2997928.0,61333504.0,4890624.0,9661.0,60162048.0,66224128.0,...,81.0,5.0,20480.0,672.0,1.614587e+09,1.000028,1.5245,2.286000e-06,25243648.0,1.612092e+09
2021-03-01 08:28:00,7.0,3315296.0,0.000015,3582216.0,3315296.0,60948480.0,5275648.0,11999.0,60702720.0,66224128.0,...,78.0,6.0,24576.0,669.0,1.614587e+09,1.000028,1.5545,2.033000e-06,25243648.0,1.612092e+09
2021-03-01 08:29:00,7.0,2975208.0,0.000015,3582216.0,2975208.0,61243392.0,4980736.0,9758.0,59785216.0,66224128.0,...,78.0,4.0,16384.0,669.0,1.614587e+09,1.000028,1.5845,1.808000e-06,25243648.0,1.612092e+09
2021-03-01 08:30:00,8.0,3211648.0,0.000015,3582216.0,3211648.0,61079552.0,5144576.0,12152.0,60473344.0,66224128.0,...,79.0,6.0,24576.0,670.0,1.614587e+09,1.000028,1.6145,1.607000e-06,25243648.0,1.612092e+09
2021-03-01 08:31:00,7.0,3257304.0,0.000015,3582216.0,3257304.0,61382656.0,4907008.0,12804.0,61325312.0,66289664.0,...,78.0,9.0,36864.0,669.0,1.614588e+09,1.000028,1.6445,1.429000e-06,25243648.0,1.612092e+09
2021-03-01 08:32:00,7.0,3117976.0,0.000015,3582216.0,3117976.0,61284352.0,4972544.0,11246.0,61153280.0,66256896.0,...,79.0,8.0,32768.0,670.0,1.614588e+09,1.000028,1.6745,1.271000e-06,25243648.0,1.612092e+09
2021-03-01 08:33:00,7.0,2776152.0,0.000015,3582216.0,2776152.0,61808640.0,4448256.0,8120.0,60153856.0,66256896.0,...,76.0,7.0,28672.0,667.0,1.614588e+09,1.000028,1.7045,1.130000e-06,25243648.0,1.612092e+09


Save the executions into files

In [9]:
metrics_T1.to_csv(path_or_buf="metrics_train/metrics_T1.csv", sep=';')
metrics_T2.to_csv(path_or_buf="metrics_train/metrics_T2.csv", sep=';')
metrics_T3.to_csv(path_or_buf="metrics_train/metrics_T3.csv", sep=';')
metrics_T4.to_csv(path_or_buf="metrics_train/metrics_T4.csv", sep=';')
metrics_T5.to_csv(path_or_buf="metrics_train/metrics_T5.csv", sep=';')
metrics_T6.to_csv(path_or_buf="metrics_train/metrics_T6.csv", sep=';')
metrics_T7.to_csv(path_or_buf="metrics_train/metrics_T7.csv", sep=';')
metrics_T8.to_csv(path_or_buf="metrics_train/metrics_T8.csv", sep=';')
metrics_T9.to_csv(path_or_buf="metrics_train/metrics_T9.csv", sep=';')
metrics_T10.to_csv(path_or_buf="metrics_train/metrics_T10.csv", sep=';')
metrics_T11.to_csv(path_or_buf="metrics_train/metrics_T11.csv", sep=';')
metrics_T12.to_csv(path_or_buf="metrics_train/metrics_T12.csv", sep=';')
metrics_T13.to_csv(path_or_buf="metrics_train/metrics_T13.csv", sep=';')
metrics_T14.to_csv(path_or_buf="metrics_train/metrics_T14.csv", sep=';')
metrics_T15.to_csv(path_or_buf="metrics_train/metrics_T15.csv", sep=';')
metrics_T16.to_csv(path_or_buf="metrics_train/metrics_T16.csv", sep=';')
metrics_T17.to_csv(path_or_buf="metrics_train/metrics_T17.csv", sep=';')
metrics_T18.to_csv(path_or_buf="metrics_train/metrics_T18.csv", sep=';')
metrics_T19.to_csv(path_or_buf="metrics_train/metrics_T19.csv", sep=';')
metrics_T20.to_csv(path_or_buf="metrics_train/metrics_T20.csv", sep=';')
metrics_T21.to_csv(path_or_buf="metrics_train/metrics_T21.csv", sep=';')
metrics_T22.to_csv(path_or_buf="metrics_train/metrics_T22.csv", sep=';')
metrics_T23.to_csv(path_or_buf="metrics_train/metrics_T23.csv", sep=';')
metrics_T24.to_csv(path_or_buf="metrics_train/metrics_T24.csv", sep=';')
metrics_T25.to_csv(path_or_buf="metrics_train/metrics_T25.csv", sep=';')
metrics_T26.to_csv(path_or_buf="metrics_train/metrics_T26.csv", sep=';')
metrics_T27.to_csv(path_or_buf="metrics_train/metrics_T27.csv", sep=';')
metrics_T28.to_csv(path_or_buf="metrics_train/metrics_T28.csv", sep=';')
metrics_T29.to_csv(path_or_buf="metrics_train/metrics_T29.csv", sep=';')
metrics_T30.to_csv(path_or_buf="metrics_train/metrics_T30.csv", sep=';')
metrics_T31.to_csv(path_or_buf="metrics_train/metrics_T31.csv", sep=';')
metrics_T32.to_csv(path_or_buf="metrics_train/metrics_T32.csv", sep=';')
metrics_T33.to_csv(path_or_buf="metrics_train/metrics_T33.csv", sep=';')
metrics_T34.to_csv(path_or_buf="metrics_train/metrics_T34.csv", sep=';')
metrics_T35.to_csv(path_or_buf="metrics_train/metrics_T35.csv", sep=';')
metrics_T36.to_csv(path_or_buf="metrics_train/metrics_T36.csv", sep=';')
metrics_T37.to_csv(path_or_buf="metrics_train/metrics_T37.csv", sep=';')
metrics_T38.to_csv(path_or_buf="metrics_train/metrics_T38.csv", sep=';')
metrics_T39.to_csv(path_or_buf="metrics_train/metrics_T39.csv", sep=';')
metrics_T40.to_csv(path_or_buf="metrics_train/metrics_T40.csv", sep=';')
